In [1]:
import jieba
import pickle
import pandas as pd
from sklearn.pipeline import Pipeline
from bert_serving.client import BertClient
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data_dir = '../train/'
features_dir = '../features/'
opscope = pd.read_csv(data_dir + 'base_info.csv',encoding='gbk')
stopwords = open(data_dir + 'cn_stopwords.txt', encoding='utf8').readlines()
stopwords = set([i.split('\n')[0] for i in stopwords])
opscope = pd.DataFrame([opscope.id, opscope.opscope]).T

In [3]:
def replace(x):
    for i in set(['，', '。', '：', '；', '（', '）', '、','*', '(', ')', '\t', '\n']):
        x = x.replace(i, '')
    return x

vocabulary = ''
for i in opscope.opscope:
    vocabulary += i

vocabulary = replace(vocabulary)
vocabulary = jieba.lcut(vocabulary)
vocabulary = set(vocabulary).difference(stopwords)
dictionary = {v : i for i, v in zip(range(len(vocabulary)), vocabulary)}

Building prefix dict from the default dictionary ...
Loading model from cache Z:\Temp\jieba.cache
Loading model cost 0.656 seconds.
Prefix dict has been built successfully.


In [4]:
def cancats(x):
    s = ''
    for i in x:
        s += i + ' '
    return s.replace('  ', ' ').replace('  ', ' ')

opscope.opscope = opscope.opscope.apply(replace)
opscope.opscope = opscope.opscope.apply(lambda x:set(jieba.lcut(x)).difference(stopwords))
opscope.opscope = opscope.opscope.apply(cancats)
corpus = list(opscope.opscope)

In [2]:
bc = BertClient()
bc.encode(['First do it', 'then do it right', 'then do it better'])

In [5]:
# vectorizer = TfidfVectorizer().fit_transform(corpus)
# pd.DataFrame([opscope.id, vectorizer])

In [6]:
opscope.to_csv(features_dir + 'opscope.csv', index=False)